In [2]:
pip install gdown pydrive tqdm tensorflow opencv-python


     ---------------------------------------- 0.0/987.4 kB ? eta -:--:--
     ---------------------------------------- 987.4/987.4 kB 8.2 MB/s  0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached google_auth-2.42.1-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached cachetools-6.2.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ----- ---------------------------------- 2.1/14.5 MB 10.1 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/14.5 MB 10.7 MB/s eta 0:00:01
   ------------------ --------------------- 6.8/14.5 MB 11.0 MB/s eta 0:00:01
   -------------------------- ------------- 9.4/14.5 MB 11.2 MB/s eta 0:00:01
   ---------------------------

  DEPRECATION: Building 'pydrive' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pydrive'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

import os
import zipfile
import shutil
import traceback
from tqdm import tqdm
import gdown
import tensorflow as tf
import cv2
import numpy as np
import random
import glob
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models, optimizers



In [4]:
FILE_ID = '1Mu69hp0vWC_a1GqnP1u3LGLqrSfKRNoy'   
OUTPUT_ZIP = 'Deepfake.zip'
LOCAL_DATA_ROOT = './deepfake_dataset_local'
UNZIP = True
FRAMES_PER_VIDEO = 6
FRAME_SIZE = (160, 160)


In [5]:
if not os.path.exists(OUTPUT_ZIP):
    print("📥 Downloading dataset from Google Drive...")
    gdown.download(f"https://drive.google.com/uc?id={FILE_ID}", OUTPUT_ZIP, quiet=False)
else:
    print("✅ Dataset ZIP already exists.")


📥 Downloading dataset from Google Drive...


Downloading...
From (original): https://drive.google.com/uc?id=1Mu69hp0vWC_a1GqnP1u3LGLqrSfKRNoy
From (redirected): https://drive.google.com/uc?id=1Mu69hp0vWC_a1GqnP1u3LGLqrSfKRNoy&confirm=t&uuid=b0c2b518-24e7-49db-a87d-46a4ff9fd0af
To: c:\Users\shail\Documents\T87-Deepfake-Video-Detection-System\Deepfake.zip
100%|██████████| 4.02G/4.02G [08:57<00:00, 7.48MB/s]


In [6]:
if UNZIP:
    print("📦 Extracting dataset...")
    os.makedirs(LOCAL_DATA_ROOT, exist_ok=True)
    try:
        with zipfile.ZipFile(OUTPUT_ZIP, 'r') as z:
            for member in tqdm(z.namelist(), desc="Extracting"):
                try:
                    z.extract(member, LOCAL_DATA_ROOT)
                except Exception as e:
                    print("Error extracting", member, ":", e)
        print("✅ Extraction complete:", LOCAL_DATA_ROOT)
    except Exception:
        traceback.print_exc()
        raise RuntimeError("Failed to unzip dataset.")


📦 Extracting dataset...


Extracting: 100%|██████████| 140003/140003 [02:56<00:00, 794.93it/s] 

✅ Extraction complete: ./deepfake_dataset_local


In [7]:
def find_dataset_root(base):
    for root, dirs, files in os.walk(base):
        if {'train','valid','test'}.issubset(set(dirs)):
            return root
        if {'real','fake'}.issubset(set(dirs)):
            return root
    return None

DATA_ROOT = find_dataset_root(LOCAL_DATA_ROOT)
if DATA_ROOT is None:
    print("Could not auto-detect dataset root. Inspect:", LOCAL_DATA_ROOT)
    print("Contents:", os.listdir(LOCAL_DATA_ROOT))
else:
    print("Detected dataset root:", DATA_ROOT)
    
for root, dirs, files in os.walk(DATA_ROOT):
    depth = root.replace(DATA_ROOT, '').count(os.sep)
    indent = ' ' * 4 * depth
    print(f"{indent}{os.path.basename(root)}/ (files: {len(files)})")
    if depth >= 2:
        continue


Detected dataset root: ./deepfake_dataset_local\real_vs_fake\real-vs-fake
real-vs-fake/ (files: 0)
    test/ (files: 0)
        fake/ (files: 10000)
        real/ (files: 10000)
    train/ (files: 0)
        fake/ (files: 50000)
        real/ (files: 50000)
    valid/ (files: 0)
        fake/ (files: 10000)
        real/ (files: 10000)
